# Preprocessing Danych Koktajli

W tym notebooku przeprowadzimy wstępne przetwarzanie danych dotyczących koktajli. Proces obejmuje:

1. Ładowanie danych z pliku JSON.
2. Czyszczenie danych poprzez usunięcie niepotrzebnych kolumn.
3. Transformację kolumn `tags` i `ingredients` na wektory binarne.
4. (Opcjonalnie) Analizę eksploracyjną danych.
5. Zapis przetworzonych danych do pliku CSV.


## Importowanie Niezbędnych Bibliotek

Zaczynamy od zaimportowania wszystkich niezbędnych bibliotek, które będą używane w procesie przetwarzania danych.


In [1]:
# Importowanie bibliotek
import json
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import warnings

# Wyłączenie FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)


## Ładowanie Danych

Wczytujemy dane z pliku `cocktail_dataset.json`, który znajduje się w katalogu `data/`. (Upewnij się, że ścieżki do plików są poprawne)


In [2]:
# Definiowanie ścieżek
BASE_DIR = Path.cwd().parent
DATA_PATH = BASE_DIR / 'data' / 'cocktail_dataset.json'
OUTPUT_PATH = BASE_DIR / 'outputs' / 'preprocessed_cocktails.csv'

# Funkcja do ładowania danych
def load_data(file_path: Path) -> list:
    """
    Wczytuje dane z pliku JSON.
    """
    with file_path.open('r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Ładowanie danych
print("Ładowanie danych...")
data = load_data(DATA_PATH)
print(f"Znaleziono {len(data)} wpisów.")


Ładowanie danych...
Znaleziono 134 wpisów.


## Podgląd Danych

Zobaczmy, jak wyglądają nasze dane przed przetwarzaniem. Wyświetlamy pierwsze kilka rekordów, aby zrozumieć strukturę danych.


In [3]:
# Podgląd pierwszych kilku rekordów
df = pd.DataFrame(data)
df.head()


,id,name,category,glass,tags,instructions,imageUrl,alcoholic,createdAt,updatedAt,ingredients
0,11000,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",Muddle mint leaves with sugar and lime juice. ...,https://cocktails.solvro.pl/images/ingredients...,1,2024-08-18T19:01:17.000+00:00,2024-08-18T19:06:16.000+00:00,"[{'id': 170, 'name': 'Soda water', 'descriptio..."
1,11001,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",Place sugar cube in old fashioned glass and sa...,https://cocktails.solvro.pl/images/ingredients...,1,2024-08-18T19:01:58.000+00:00,2024-08-18T19:06:17.000+00:00,"[{'id': 513, 'name': 'Water', 'description': '..."
2,11002,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",Combine all ingredients (except cola) and pour...,https://cocktails.solvro.pl/images/ingredients...,1,2024-08-18T19:01:58.000+00:00,2024-08-18T19:06:17.000+00:00,"[{'id': 305, 'name': 'Light Rum', 'description..."
3,11003,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic]","Stir into glass over ice, garnish and serve.",https://cocktails.solvro.pl/images/ingredients...,1,2024-08-18T19:01:58.000+00:00,2024-08-18T19:06:17.000+00:00,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri..."
4,11004,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]","Shake with ice. Strain into chilled glass, gar...",https://cocktails.solvro.pl/images/ingredients...,1,2024-08-18T19:01:59.000+00:00,2024-08-18T19:06:18.000+00:00,"[{'id': 409, 'name': 'Powdered Sugar', 'descri..."


## Czyszczenie Danych

Usuńmy niepotrzebne kolumny (`createdAt`, `updatedAt`, `instructions`, `imageUrl`, `id`) oraz przekształćmy kolumny `tags` i `ingredients` na listy nazw. Dzięki temu uprościmy nasze dane, przygotowując je do dalszej analizy.


In [4]:
# Lista pól do wykluczenia
EXCLUDED_FIELDS = ['createdAt', 'updatedAt', 'instructions', 'imageUrl', 'id']

# Funkcja do przetwarzania pojedynczego wpisu
def preprocess_entry(entry: dict) -> dict:
    """
    Przetwarza pojedynczy wpis z danych i usuwa niechciane pola.
    """
    # Usunięcie niechcianych pól
    for field in EXCLUDED_FIELDS:
        entry.pop(field, None)
    
    # Przekształcenie list na listy nazw
    entry['tags'] = entry['tags'] if entry.get('tags') else []
    entry['ingredients'] = [ingredient['name'] for ingredient in entry.get('ingredients', [])]
    
    return entry

# Przetwarzanie wszystkich wpisów
print("Czyszczenie danych...")
processed_data = [preprocess_entry(entry) for entry in data]
df_clean = pd.DataFrame(processed_data)
df_clean.head()


Czyszczenie danych...


,name,category,glass,tags,alcoholic,ingredients
0,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",1,"[Soda water, Light Rum, Lime, Mint, Sugar]"
1,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",1,"[Water, Sugar, Angostura Bitters, Bourbon]"
2,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",1,"[Light Rum, lemon, Vodka, Gin, Tequila, Coca-C..."
3,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic]",1,"[Sweet Vermouth, Gin, Campari]"
4,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]",1,"[Powdered Sugar, lemon, Blended Whiskey, Cherry]"


## Transformacja `tags` i `ingredients` na Wektory

Aby umożliwić efektywną analizę i modelowanie danych, przekształcamy kolumny `tags` i `ingredients` na wektory binarne. Używamy do tego `MultiLabelBinarizer` z biblioteki `scikit-learn`.

- **`tag_vector`**: Binarna reprezentacja obecności każdego tagu.
- **`ingredient_vector`**: Binarna reprezentacja obecności każdego składnika.

Po przekształceniu, oryginalne kolumny `tags` i `ingredients` są usuwane, a ich miejsce zajmują nowe kolumny wektorowe.


In [5]:
# Funkcja do tworzenia wektorów binarnych
def create_binary_vectors(df: pd.DataFrame, column: str) -> tuple:
    """
    Tworzy wektor binarny dla danej kolumny z listami.
    """
    mlb = MultiLabelBinarizer()
    binary_matrix = mlb.fit_transform(df[column])
    binary_vectors = binary_matrix.tolist()
    return binary_vectors, mlb.classes_

# Tworzenie wektorów tagów
print("Tworzenie wektorów tagów...")
df_clean['tag_vector'], tag_classes = create_binary_vectors(df_clean, 'tags')

# Tworzenie wektorów składników
print("Tworzenie wektorów składników...")
df_clean['ingredient_vector'], ingredient_classes = create_binary_vectors(df_clean, 'ingredients')

# Usunięcie oryginalnych kolumn 'tags' i 'ingredients'
df_final = df_clean.drop(['tags', 'ingredients'], axis=1)

# Podgląd przetworzonych danych
df_final.head()


Tworzenie wektorów tagów...
Tworzenie wektorów składników...


,name,category,glass,alcoholic,tag_vector,ingredient_vector
0,Mojito,Cocktail,Highball glass,1,"[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Old Fashioned,Cocktail,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,Long Island Tea,Ordinary Drink,Highball glass,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Negroni,Ordinary Drink,Old-fashioned glass,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Whiskey Sour,Ordinary Drink,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


## Normalizacja Danych

Po przekształceniu kolumn `tags` i `ingredients` na wektory binarne, przeprowadzimy normalizację danych. Normalizacja pozwala na skalowanie cech do określonego zakresu, co jest istotne dla wielu algorytmów uczenia maszynowego.


In [6]:
import numpy as np

# Funkcja do obliczania normy wektora
def calculate_norm(vector):
    return np.linalg.norm(vector)

# Obliczanie normy dla tag_vector
print("Obliczanie normy tag_vector...")
df_final['tag_vector_norm'] = df_final['tag_vector'].apply(lambda x: calculate_norm(x))

# Obliczanie normy dla ingredient_vector
print("Obliczanie normy ingredient_vector...")
df_final['ingredient_vector_norm'] = df_final['ingredient_vector'].apply(lambda x: calculate_norm(x))

# Podgląd przetworzonych danych po normalizacji
df_final.head()


Obliczanie normy tag_vector...
Obliczanie normy ingredient_vector...


,name,category,glass,alcoholic,tag_vector,ingredient_vector,tag_vector_norm,ingredient_vector_norm
0,Mojito,Cocktail,Highball glass,1,"[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3.162278,2.236068
1,Old Fashioned,Cocktail,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",2.236068,2.000000
2,Long Island Tea,Ordinary Drink,Highball glass,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.449490,2.449490
3,Negroni,Ordinary Drink,Old-fashioned glass,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.414214,1.732051
4,Whiskey Sour,Ordinary Drink,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",2.000000,2.000000


## One-Hot Encoding Kategorii

Przekształcamy kolumnę `category` na wektory binarne (`category_vector`), gdzie każda pozycja w wektorze odpowiada konkretnej kategorii. Na przykład, jeśli mamy trzy kategorie: `Cocktail`, `Shot`, `Punch`, to wektor `[1, 0, 0]` będzie oznaczał `Cocktail`, `[0, 1, 0]` - `Shot`, a `[0, 0, 1]` - `Punch`.


In [7]:
from sklearn.preprocessing import LabelEncoder

# Funkcja do tworzenia wartości liczbowych dla kategorii
def create_category_labels(df: pd.DataFrame, column: str) -> tuple:
    """
    Tworzy wartości liczbowe dla danej kolumny kategorii.
    """
    # Sprawdzenie wersji scikit-learn
    import sklearn
    sklearn_version = sklearn.__version__
    print(f"Używana wersja scikit-learn: {sklearn_version}")

    encoder = LabelEncoder()

    category_vector = encoder.fit_transform(df[column])
    return category_vector, encoder.classes_

# Tworzenie wartości liczbowych dla kategorii
print("Tworzenie wartości liczbowych dla kategorii...")
df_final['category_vector'], category_classes = create_category_labels(df_final, 'category')

# Podgląd przetworzonych danych po kodowaniu etykietami
df_final.head()


Tworzenie wartości liczbowych dla kategorii...
Używana wersja scikit-learn: 1.5.2


,name,category,glass,alcoholic,tag_vector,ingredient_vector,tag_vector_norm,ingredient_vector_norm,category_vector
0,Mojito,Cocktail,Highball glass,1,"[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3.162278,2.236068,0
1,Old Fashioned,Cocktail,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",2.236068,2.000000,0
2,Long Island Tea,Ordinary Drink,Highball glass,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.449490,2.449490,1
3,Negroni,Ordinary Drink,Old-fashioned glass,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.414214,1.732051,1
4,Whiskey Sour,Ordinary Drink,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",2.000000,2.000000,1


## One-Hot Encoding `glass`

Przekształcamy kolumnę `glass` na wektory binarne (`glass_vector`), gdzie każda pozycja w wektorze odpowiada konkretnego typu szkła użytego do serwowania drinka. Na przykład, jeśli mamy trzy typy szkła: `Highball glass`, `Cocktail glass`, `Shot glass`, to wektor `[1, 0, 0]` będzie oznaczał `Highball glass`, `[0, 1, 0]` - `Cocktail glass`, a `[0, 0, 1]` - `Shot glass`.


In [8]:
from sklearn.preprocessing import OneHotEncoder

# Funkcja do tworzenia wektorów binarnych dla kolumny 'glass'
def create_glass_vector(df: pd.DataFrame, column: str) -> tuple:
    """
    Tworzy wektor binarny dla danej kolumny 'glass'.
    """
    # Sprawdzenie wersji scikit-learn
    import sklearn
    sklearn_version = sklearn.__version__
    print(f"Używana wersja scikit-learn: {sklearn_version}")
    
    encoder = OneHotEncoder(sparse_output=False)
    
    glass_matrix = encoder.fit_transform(df[[column]])
    glass_vectors = glass_matrix.tolist()
    return glass_vectors, encoder.categories_[0]

# Tworzenie wektorów 'glass'
print("Tworzenie wektorów 'glass'...")
df_final['glass_vector'], glass_classes = create_glass_vector(df_final, 'glass')

# Podgląd przetworzonych danych po one-hot encoding dla 'glass'
df_final.head()


Tworzenie wektorów 'glass'...
Używana wersja scikit-learn: 1.5.2


,name,category,glass,alcoholic,tag_vector,ingredient_vector,tag_vector_norm,ingredient_vector_norm,category_vector,glass_vector
0,Mojito,Cocktail,Highball glass,1,"[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3.162278,2.236068,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,Old Fashioned,Cocktail,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",2.236068,2.000000,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,Long Island Tea,Ordinary Drink,Highball glass,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.449490,2.449490,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,Negroni,Ordinary Drink,Old-fashioned glass,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.414214,1.732051,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,Whiskey Sour,Ordinary Drink,Old-fashioned glass,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",2.000000,2.000000,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


## Redukcja Kolumn i Zapis Przetworzonych Danych

Po przetworzeniu danych, zachowamy tylko kluczowe kolumny:
- `name`
- `tag_vector_norm`
- `ingredient_vector_norm`
- `category_vector`
- `glass_vector`

Następnie zapisujemy ten zmniejszony DataFrame do pliku CSV, co umożliwi łatwe wykorzystanie danych w dalszych etapach analizy lub modelowania.


In [9]:
# Definicja listy kolumn do zachowania
columns_to_keep = ['name', 'tag_vector_norm', 'ingredient_vector_norm', 'category_vector', 'glass_vector']

# Sprawdzenie, czy wszystkie wymagane kolumny istnieją w DataFrame
missing_columns = [col for col in columns_to_keep if col not in df_final.columns]
if missing_columns:
    raise ValueError(f"Brak wymaganych kolumn w DataFrame: {missing_columns}")

# Wybór tylko żądanych kolumn
df_final_selected = df_final[columns_to_keep]

# Wyświetlenie pierwszych kilku wierszy zmniejszonego DataFrame
df_final_selected.head()


,name,tag_vector_norm,ingredient_vector_norm,category_vector,glass_vector
0,Mojito,3.162278,2.236068,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,Old Fashioned,2.236068,2.000000,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,Long Island Tea,2.449490,2.449490,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,Negroni,1.414214,1.732051,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,Whiskey Sour,2.000000,2.000000,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


## Analiza Eksploracyjna Danych (Opcjonalnie)

Przeprowadzimy kilka podstawowych analiz, aby lepiej zrozumieć nasze dane.


In [10]:
# Najczęstsze tagi
tag_counts = pd.Series([tag for tags in df_clean['tags'] for tag in tags]).value_counts()
top_10_tags = tag_counts.head(10)
print("Top 10 najczęstszych tagów:")
print(top_10_tags)


Top 10 najczęstszych tagów:
IBA                    23
ContemporaryClassic    13
Classic                 9
Alcoholic               6
Brunch                  3
Beach                   3
Citrus                  2
Hangover                2
Dairy                   2
Sour                    2
Name: count, dtype: int64


In [11]:
# Najczęstsze składniki
ingredient_counts = pd.Series([ingredient for ingredients in df_clean['ingredients'] for ingredient in ingredients]).value_counts()
top_10_ingredients = ingredient_counts.head(10)
print("Top 10 najczęstszych składników:")
print(top_10_ingredients)


Top 10 najczęstszych składników:
Gin               45
Light Rum         20
Triple Sec        20
Lemon Juice       19
Sugar             19
lemon             17
Powdered Sugar    17
Lemon Peel        17
Lime              15
Dry Vermouth      14
Name: count, dtype: int64


## Zapis Przetworzonych Danych

Na zakończenie zapisujemy przetworzone dane do plików w katalogu `outputs/`. Dzięki temu możemy je łatwo wykorzystać w dalszych etapach analizy lub modelowania.
